In [1]:
from bs4 import BeautifulSoup 

import bs4 as bs
import pandas as pd
import requests 

In [2]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
nh_toronto = pd.read_html(str(table))[0] # must have [0], not sure why 
   

nh_toronto.head()
     



,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# clean the data
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df = nh_toronto[nh_toronto.Borough !='Not assigned']
df.head()
df.iloc[5:9,]

,Postcode,Borough,Neighbourhood
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge


In [4]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#df['Neighbourhood'].apply(lambda x: df.Borough if x=='Not assigned')

df.loc[(df.Neighbourhood == 'Not assigned'),'Neighbourhood']=df.Borough
df.iloc[5:9,]

/Users/wangshuang/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/wangshuang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Postcode,Borough,Neighbourhood
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge


In [5]:
# More than one neighborhood in one postal code area will be combined into one row with the neighborhoods separated with a comma 
# reference: https://stackoverflow.com/questions/36274014/combine-rows-of-multiindex-dataframe-into-comma-separated-lists
df.set_index(['Postcode','Borough'],inplace=True)
df1 = df.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
df1.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [6]:
df1.reset_index(inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


explanation and assumptions:
1. scrape the table from the website using Beautifulsoup
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
3. Replace the "Not assigned" neighborhood with the name of the Borough.
to do this, first we need to set index with Borough and Neighborhood, and remove index afterwards. 



In [7]:
df1.shape

(103, 3)

In [18]:
# obtain the geographical coordinates of each postal code
postcode = pd.read_csv('Geospatial_Coordinates.csv')
postcode.head()
df2 = pd.merge(df1, postcode, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)
df2

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [16]:
# starting segmenting and clustering 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/wangshuang/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         143 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h01d97ff_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geop

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/info/test/tests/data/conda_format_repo/index.html.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/info/test/tests/data/conda_format_repo/channeldata.json.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/info/test/tests/data/build-index2-json.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/info/test/tests/data/

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/info/index.json.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/info/git.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/core/__pycache__/portability.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/core/__pycache__/solve.cpython-3

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/_vendor/urllib3/exceptions.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/_vendor/auxlib/_vendor/boltons/__pycache__/timeutils.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/_vendor/auxlib/_vendor/boltons/__pycache__/__init__.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rena

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/models/__pycache__/version.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/models/__pycache__/records.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/models/__pycache__/package_info.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove

...

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/cli/__pycache__/__init__.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/cli/__pycache__/main_package.cpython-37.pyc.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/cli/main_clean.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/gateways/disk/update.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/gateways/disk/link.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda/gateways/disk/delete.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda_env/__main__.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/conda-4.7.12-py37_0/lib/python3.7/site-packages/conda_env/pip_util.py.  Please remove this file manually (you may need to reboot to free file handles)
conda-4.7.12         | 3.0 MB    | ##################################### | 100% 
certifi-2019.9.11    | 147 KB    |                                       |   0% WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/certifi-2019.9.11-py37_0/info/test/run_test.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/certifi-2019.9.11-py37_0/info/test/run_test.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/certifi-2019.9.11-py37_0/info/licenses/certifi/LICENSE.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/certifi-2019.9.11-py37_0/info/recipe/recipe_log.txt.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /Users/wangshuang/anaconda3/pkgs/certifi-2019.9.11-py37_0/info/recipe/bld.bat.  Please remove this file manually (you may need t

ModuleNotFoundError: No module named 'geopy'

start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [17]:
CLIENT_ID = 'BIJTGC301CNCHK2AMQHUK2M33E45A00LQH4DKYHVICH2Y0IF' # your Foursquare ID
CLIENT_SECRET = 'XY1NCASPNMQ05WWUE054LRR2R5W010HHCPDFGCDOERYNKSVH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BIJTGC301CNCHK2AMQHUK2M33E45A00LQH4DKYHVICH2Y0IF
CLIENT_SECRET:XY1NCASPNMQ05WWUE054LRR2R5W010HHCPDFGCDOERYNKSVH


In [22]:
# analyze only the Borough of North York
df3 = df2.loc[df2.Borough=='North York',:]
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 72
Data columns (total 5 columns):
Postcode         24 non-null object
Borough          24 non-null object
Neighbourhood    24 non-null object
Latitude         24 non-null float64
Longitude        24 non-null float64
dtypes: float64(2), object(3)
memory usage: 1.1+ KB


Let's create a function to explore the neighborhood in North York 

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now write the code to run the above function on each neighborhood and create a new dataframe called northyork_venues.

In [66]:
northyork_venues = getNearbyVenues(names=df3['Neighbourhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )
print(northyork_venues.shape)
northyork_venues.head()

Parkwoods
Victoria Village
Lawrence Heights,Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park,Don Mills South
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Fairview,Henry Farm,Oriole
Northwood Park,York University
Bayview Village
CFB Toronto,Downsview East
Silver Hills,York Mills
Downsview West
Downsview,North Park,Upwood Park
Humber Summit
Newtonbrook,Willowdale
Downsview Central
Bedford Park,Lawrence Manor East
Emery,Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West
(241, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [67]:
# Let's check how many venues were returned for each neighborhood
northyork_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
"CFB Toronto,Downsview East",2,2,2,2,2,2
Don Mills North,5,5,5,5,5,5
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,5,5,5,5,5,5
"Downsview,North Park,Upwood Park",4,4,4,4,4,4


In [68]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 105 uniques categories.


## 3. Analyze Each Neighborhood

In [69]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighbourhood'] = northyork_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
manhattan_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()
northyork_onehot.shape

(241, 106)

In [70]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
northyork_grouped = northyork_onehot.groupby('Neighbourhood').mean().reset_index()
northyork_grouped
northyork_grouped.shape

(23, 106)

In [71]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in northyork_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
            venue  freq
0     Coffee Shop  0.11
1        Pharmacy  0.05
2      Restaurant  0.05
3  Sandwich Place  0.05
4   Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.09
1  Fast Food Restaurant  0.09
2           Coffee Shop  0.09
3         Grocery Store  0.04
4               Butcher  0.04


----CFB Toronto,Downsview East----
         venue  freq
0      Airport   0.5
1         Park   0.5
2  Pizza Place   0.0
3     Pharmacy   0.0
4    Pet Store   0.0


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2   Japanese Restaurant   0.2
3                  Café   0.2
4      Basketball Court   0.2


----Down

In [91]:
# a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = northyork_grouped['Neighbourhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Bank,Fried Chicken Joint,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park,Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant
3,"CFB Toronto,Downsview East",Airport,Park,Women's Store,Electronics Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Basketball Court,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop


## 4. Cluster Neighborhoods

In [92]:
# Run k-means to cluster the neighborhood into 4 clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighbourhood', 1)

# run k-means clustering

# import k-means from clustering stage
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([0, 0, 0, 4, 0, 0, 0, 0, 4, 3, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0,
       0], dtype=int32)

In [93]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

northyork_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,BBQ Joint,Food & Drink Shop,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Accessories Store,Boutique,Event Space,Miscellaneous Shop,Coffee Shop,Clothing Store,Gift Shop,Vietnamese Restaurant,Athletics & Sports
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Basketball Court,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop
10,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Pizza Place,Asian Restaurant,Pub,Japanese Restaurant,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
13,M3C,North York,"Flemingdon Park,Don Mills South",43.725900,-79.340923,0.0,Coffee Shop,Asian Restaurant,Gym,Beer Store,Bike Shop,Sporting Goods Shop,Italian Restaurant,Restaurant,Discount Store,Sandwich Place
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Dog Run,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
28,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259,0.0,Coffee Shop,Frozen Yogurt Shop,Bank,Fried Chicken Joint,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner
33,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Asian Restaurant,Bakery,Chinese Restaurant,Toy / Game Store,Japanese Restaurant,Frozen Yogurt Shop
34,M3J,North York,"Northwood Park,York University",43.767980,-79.487262,0.0,Coffee Shop,Caribbean Restaurant,Miscellaneous Shop,Falafel Restaurant,Bar,Massage Studio,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store


In [97]:

latitude = 43.7615   # from google 
longitude = -79.4111


# create map
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map

northyork_merged = northyork_merged.dropna()

markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighbourhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters